In [ ]:
import pandas as pd # type: ignore
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set_theme(style="whitegrid")
import os
import glob as gb
import cv2
import tensorflow as tf
import keras
from keras.callbacks import ModelCheckpoint 
from keras.callbacks import TensorBoard
from keras.callbacks import EarlyStopping
import pickle


In [ ]:
### dataset paths

dataset = 'Dataset'
trainpath = 'HMBD v1 dataset/trainpath'
testpath = 'HMBD v1 dataset/testpath'
predpath = 'HMBD v1 dataset/predpath'
md_file_path = 'info.md'

In [ ]:
code = {
    'Ain_End': 0,
    'Ain_Isolated': 1,
    'Ain_Middle': 2,
    'Ain_Start': 3,
    'Alf_End': 4,
    'Alf_Hamza_Above_End': 5,
    'Alf_Hamza_Above_Isolated': 6,
    'Alf_Hamza_Under_End': 7,
    'Alf_Hamza_Under_Isolated': 8,
    'Alf_Isolated': 9,
    'Baa_End': 10,
    'Baa_Isolated': 11,
    'Baa_Middle': 12,
    'Baa_Start': 13,
    'Daad_End': 14,
    'Daad_Isolated': 15,
    'Daad_Middle': 16,
    'Daad_Start': 17,
    'Dal_End': 18,
    'Dal_Isolated': 19,
    'Faa_End': 20,
    'Faa_Isolated': 21,
    'Faa_Middle': 22,
    'Faa_Start': 23,
    'Gem_End': 24,
    'Gem_Isolated': 25,
    'Gem_Start': 26,
    'Gen_End': 27,
    'Gen_Isolated': 28,
    'Gen_Middle': 29,
    'Ha_End': 30,
    'Ha_Isolated': 31,
    'Ha_Middle': 32,
    'Ha_Start': 33,
    'Haa_End': 34,
    'Haa_Isolated': 35,
    'Haa_Start': 36,
    'Hamza_Isolated': 37,
    'Kaf_End': 38,
    'Kaf_Isolated': 39,
    'Kaf_Middle': 40,
    'Kaf_Start': 41,
    'Khaa_End': 42,
    'Khaa_Isolated': 43,
    'Khaa_Start': 44,
    'Lam_Alf_End': 45,
    'Lam_Alf_Hamza_End': 46,
    'Lam_Alf_Hamza_Isolated': 47,
    'Lam_Alf_Isolated': 48,
    'Lam_Alf_Mad_End': 49,
    'Lam_Alf_Mad_Isolated': 50,
    'Lam_End': 51,
    'Lam_Isolated': 52,
    'Lam_Middle': 53,
    'Lam_Start': 54,
    'Mem_End': 55,
    'Mem_Isolated': 56,
    'Mem_Start': 57,
    'Non_End': 58,
    'Non_Isolated': 59,
    'Non_Middle': 60,
    'Non_Start': 61,
    'Qaf_End': 62,
    'Qaf_Isolated': 63,
    'Qaf_Middle': 64,
    'Qaf_Start': 65,
    'Raa_End': 66,
    'Raa_Isolated': 67,
    'Saad_End': 68,
    'Saad_Isolated': 69,
    'Saad_Middle': 70,
    'Saad_Start': 71,
    'Shen_End': 72,
    'Shen_Isolated': 73,
    'Shen_Middle': 74,
    'Shen_Start': 75,
    'Sin_End': 76,
    'Sin_Isolated': 77,
    'Sin_Middle': 78,
    'Sin_Start': 79,
    'Taa_End': 80,
    'Taa_Isolated': 81,
    'Taa_Middle': 82,
    'Taa_Start': 83,
    'Tah_Isolated': 84,
    'Tah_Middle': 85,
    'Thaa_End': 86,
    'Thaa_Isolated': 87,
    'Thaa_Middle': 88,
    'Thaa_Start': 89,
    'Waw_End': 90,
    'Waw_Hamza_Isolated': 91,
    'Waw_Isolated': 92,
    'Yaa_Dot_End': 93,
    'Yaa_Dot_Isolated': 94,
    'Yaa_Dot_Start': 95,
    'Yaa_End': 96,
    'Yaa_Isolated': 97,
    'Yaa_Middle': 98,
    'Zah_Isolated': 99,
    'Zah_Middle': 100,
    'Zal_End': 101,
    'Zal_Isolated': 102,
    'Zin_End': 103,
    'Zin_Isolated': 104
}



def getcode(n) : 
    for x , y in code.items() : 
        if n == y : 
            return x    

In [ ]:
s=64


In [ ]:


pickle_in=open("dataSave/x_train.pickle","rb")
X_train= pickle.load(pickle_in)


pickle_in=open("dataSave/y_train.pickle","rb")
y_train= pickle.load(pickle_in)
 

pickle_in=open("dataSave/x_test.pickle","rb")
X_test= pickle.load(pickle_in)


pickle_in=open("dataSave/y_test.pickle","rb")
y_test= pickle.load(pickle_in)

pickle_in=open("dataSave/X_pred_array.pickle","rb")
X_pred_array= pickle.load(pickle_in)


In [ ]:

# checkpoint_filepath='weights-improvement--{epoch:02d}-{val_accuray:.2f}.hdf5' #لعمل عددة ملفات حسب الدقة والايبوك
checkpoint_filepath='dataSave/weights.best.hdf5' # or 'best_model.hdf5' any name
checkpoint=ModelCheckpoint(filepath=checkpoint_filepath,monitor='val_accuracy',verbose=1,save_best_only=True,mode='max')
tensorBoard=TensorBoard(log_dir='dataSave/model_logs',histogram_freq=1,write_graph=True,write_images=True) 
es=EarlyStopping(monitor='val_loss',mode='min',verbose=1,patience=10) # اذا وصلت الى اقل قيمة للوس زيد 10 دورات اذا قلت قيمة اللوس كمل دورات التدريب واذا زادت قيمة اللوس وقف التدريب

callbacks_list=[checkpoint,tensorBoard,es]


 

In [ ]:
KerasModel = keras.models.Sequential([
    keras.layers.Conv2D(16, kernel_size=(16, 16), activation='relu', input_shape=(s, s, 3)),
    keras.layers.Conv2D(16, kernel_size=(16, 16), activation='relu'),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPool2D(pool_size=(2, 2), strides=2), 
    
    keras.layers.Conv2D(32, kernel_size=(8, 8), activation='relu'),
    keras.layers.Conv2D(48, kernel_size=(6, 6), activation='relu'),
    keras.layers.BatchNormalization(),  
    keras.layers.MaxPool2D(pool_size=(2, 2), strides=1), 
    keras.layers.Flatten(), #768 
    keras.layers.Dense(512, activation='relu'),
    keras.layers.Dense(105, activation='softmax')
])

In [ ]:
KerasModel.compile(optimizer ='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])
print('Model Details are : ')
print(KerasModel.summary())

In [ ]:
epochs = 2
# history=KerasModel.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=epochs,verbose=1,callbacks=callbacks_list)

ThisModel = KerasModel.fit(X_train, y_train, epochs=epochs,batch_size=64,callbacks=callbacks_list,verbose=1)



In [ ]:
ModelLoss, ModelAccuracy = KerasModel.evaluate(X_test, y_test)

print('Test Loss is {}'.format(ModelLoss))
print('Test Accuracy is {}'.format(ModelAccuracy ))

In [ ]:
y_pred = KerasModel.predict(X_test)

print('Prediction Shape is {}'.format(y_pred.shape))

In [ ]:
y_result = KerasModel.predict(X_pred_array)

print('Prediction Shape is {}'.format(y_result.shape))

In [ ]:
plt.figure(figsize=(12,12))
for n , i in enumerate(list(np.random.randint(0,len(X_pred),36))) : 
    plt.subplot(6,6,n+1)
    plt.imshow(X_pred_array[i])    
    plt.axis('off')
    plt.title(getcode(np.argmax(y_result[i])))